## Chatbot Interaktif Phi 3 Mini 4K Instruct dengan Whisper

### Pendahuluan:
Chatbot Interaktif Phi 3 Mini 4K Instruct adalah alat yang memungkinkan pengguna berinteraksi dengan demo Microsoft Phi 3 Mini 4K Instruct menggunakan input teks atau audio. Chatbot ini dapat digunakan untuk berbagai tugas, seperti penerjemahan, pembaruan cuaca, dan pengumpulan informasi umum.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


Buat Token Akses Huggingface Anda

Buat token baru  
Berikan nama baru  
Pilih izin tulis  
Salin token dan simpan di tempat yang aman


Python berikut melakukan dua tugas utama: mengimpor modul `os` dan menetapkan variabel lingkungan.

1. Mengimpor modul `os`:
   - Modul `os` dalam Python menyediakan cara untuk berinteraksi dengan sistem operasi. Modul ini memungkinkan Anda melakukan berbagai tugas terkait sistem operasi, seperti mengakses variabel lingkungan, bekerja dengan file dan direktori, dll.
   - Dalam kode ini, modul `os` diimpor menggunakan pernyataan `import`. Pernyataan ini membuat fungsi dari modul `os` tersedia untuk digunakan dalam skrip Python saat ini.

2. Menetapkan variabel lingkungan:
   - Variabel lingkungan adalah nilai yang dapat diakses oleh program yang berjalan di sistem operasi. Ini adalah cara untuk menyimpan pengaturan konfigurasi atau informasi lain yang dapat digunakan oleh beberapa program.
   - Dalam kode ini, variabel lingkungan baru ditetapkan menggunakan kamus `os.environ`. Kunci dari kamus adalah `'HF_TOKEN'`, dan nilainya diberikan dari variabel `HUGGINGFACE_TOKEN`.
   - Variabel `HUGGINGFACE_TOKEN` didefinisikan tepat di atas cuplikan kode ini, dan diberi nilai string `"hf_**************"` menggunakan sintaks `#@param`. Sintaks ini sering digunakan dalam Jupyter notebook untuk memungkinkan input pengguna dan konfigurasi parameter langsung di antarmuka notebook.
   - Dengan menetapkan variabel lingkungan `'HF_TOKEN'`, variabel ini dapat diakses oleh bagian lain dari program atau program lain yang berjalan di sistem operasi yang sama.

Secara keseluruhan, kode ini mengimpor modul `os` dan menetapkan variabel lingkungan bernama `'HF_TOKEN'` dengan nilai yang diberikan dalam variabel `HUGGINGFACE_TOKEN`.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Cuplikan kode ini mendefinisikan sebuah fungsi bernama clear_output yang digunakan untuk menghapus output dari sel saat ini di Jupyter Notebook atau IPython. Mari kita uraikan kode ini dan pahami fungsinya:

Fungsi clear_output memiliki satu parameter bernama wait, yang merupakan nilai boolean. Secara default, wait diatur ke False. Parameter ini menentukan apakah fungsi harus menunggu hingga output baru tersedia untuk menggantikan output yang ada sebelum menghapusnya.

Fungsi ini sendiri digunakan untuk menghapus output dari sel saat ini. Di Jupyter Notebook atau IPython, ketika sebuah sel menghasilkan output, seperti teks yang dicetak atau grafik, output tersebut ditampilkan di bawah sel. Fungsi clear_output memungkinkan Anda untuk menghapus output tersebut.

Implementasi fungsi ini tidak disediakan dalam cuplikan kode, seperti yang ditunjukkan oleh tanda elipsis (...). Tanda elipsis mewakili placeholder untuk kode aktual yang melakukan penghapusan output. Implementasi fungsi ini mungkin melibatkan interaksi dengan API Jupyter Notebook atau IPython untuk menghapus output yang ada dari sel.

Secara keseluruhan, fungsi ini menyediakan cara yang praktis untuk menghapus output dari sel saat ini di Jupyter Notebook atau IPython, sehingga mempermudah pengelolaan dan pembaruan output yang ditampilkan selama sesi pengkodean interaktif.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Melakukan text-to-speech (TTS) menggunakan layanan Edge TTS. Mari kita bahas implementasi fungsi yang relevan satu per satu:

1. `calculate_rate_string(input_value)`: Fungsi ini menerima nilai input dan menghitung string kecepatan untuk suara TTS. Nilai input mewakili kecepatan bicara yang diinginkan, di mana nilai 1 mewakili kecepatan normal. Fungsi ini menghitung string kecepatan dengan mengurangi 1 dari nilai input, mengalikannya dengan 100, dan kemudian menentukan tanda berdasarkan apakah nilai input lebih besar atau sama dengan 1. Fungsi ini mengembalikan string kecepatan dalam format "{sign}{rate}".

2. `make_chunks(input_text, language)`: Fungsi ini menerima teks input dan bahasa sebagai parameter. Fungsi ini membagi teks input menjadi beberapa bagian berdasarkan aturan spesifik bahasa. Dalam implementasi ini, jika bahasanya adalah "English", fungsi ini membagi teks di setiap titik (".") dan menghapus spasi di awal atau akhir. Kemudian, fungsi ini menambahkan titik ke setiap bagian dan mengembalikan daftar bagian yang telah difilter.

3. `tts_file_name(text)`: Fungsi ini menghasilkan nama file untuk file audio TTS berdasarkan teks input. Fungsi ini melakukan beberapa transformasi pada teks: menghapus titik di akhir (jika ada), mengubah teks menjadi huruf kecil, menghapus spasi di awal dan akhir, serta mengganti spasi dengan garis bawah. Kemudian, fungsi ini memotong teks hingga maksimal 25 karakter (jika lebih panjang) atau menggunakan teks penuh jika kosong. Akhirnya, fungsi ini menghasilkan string acak menggunakan modul [`uuid`] dan menggabungkannya dengan teks yang telah dipotong untuk membuat nama file dalam format "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: Fungsi ini menggabungkan beberapa file audio menjadi satu file audio. Fungsi ini menerima daftar jalur file audio dan jalur output sebagai parameter. Fungsi ini menginisialisasi objek kosong `AudioSegment` yang disebut [`merged_audio`]. Kemudian, fungsi ini melakukan iterasi melalui setiap jalur file audio, memuat file audio menggunakan metode `AudioSegment.from_file()` dari pustaka `pydub`, dan menambahkan file audio saat ini ke objek [`merged_audio`]. Akhirnya, fungsi ini mengekspor audio yang telah digabungkan ke jalur output yang ditentukan dalam format MP3.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: Fungsi ini melakukan operasi TTS menggunakan layanan Edge TTS. Fungsi ini menerima daftar bagian teks, kecepatan bicara, nama suara, dan jalur penyimpanan sebagai parameter. Jika jumlah bagian lebih dari 1, fungsi ini membuat direktori untuk menyimpan file audio bagian individu. Kemudian, fungsi ini melakukan iterasi melalui setiap bagian, menyusun perintah Edge TTS menggunakan fungsi `calculate_rate_string()`, nama suara, dan teks bagian, lalu menjalankan perintah menggunakan fungsi `os.system()`. Jika eksekusi perintah berhasil, fungsi ini menambahkan jalur file audio yang dihasilkan ke daftar. Setelah memproses semua bagian, fungsi ini menggabungkan file audio individu menggunakan fungsi `merge_audio_files()` dan menyimpan audio yang telah digabungkan ke jalur penyimpanan yang ditentukan. Jika hanya ada satu bagian, fungsi ini langsung menghasilkan perintah Edge TTS dan menyimpan audio ke jalur penyimpanan. Akhirnya, fungsi ini mengembalikan jalur penyimpanan file audio yang dihasilkan.

6. `random_audio_name_generate()`: Fungsi ini menghasilkan nama file audio acak menggunakan modul [`uuid`]. Fungsi ini menghasilkan UUID acak, mengonversinya menjadi string, mengambil 8 karakter pertama, menambahkan ekstensi ".mp3", dan mengembalikan nama file audio acak.

7. `talk(input_text)`: Fungsi ini adalah titik masuk utama untuk melakukan operasi TTS. Fungsi ini menerima teks input sebagai parameter. Fungsi ini pertama-tama memeriksa panjang teks input untuk menentukan apakah itu kalimat panjang (lebih besar atau sama dengan 600 karakter). Berdasarkan panjang dan nilai variabel `translate_text_flag`, fungsi ini menentukan bahasa dan menghasilkan daftar bagian teks menggunakan fungsi `make_chunks()`. Kemudian, fungsi ini menghasilkan jalur penyimpanan untuk file audio menggunakan fungsi `random_audio_name_generate()`. Akhirnya, fungsi ini memanggil fungsi `edge_free_tts()` untuk melakukan operasi TTS dan mengembalikan jalur penyimpanan file audio yang dihasilkan.

Secara keseluruhan, fungsi-fungsi ini bekerja sama untuk membagi teks input menjadi bagian-bagian, menghasilkan nama file untuk file audio, melakukan operasi TTS menggunakan layanan Edge TTS, dan menggabungkan file audio individu menjadi satu file audio.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

Implementasi dua fungsi: convert_to_text dan run_text_prompt, serta deklarasi dua kelas: str dan Audio.

Fungsi convert_to_text menerima audio_path sebagai input dan mentranskripsi audio menjadi teks menggunakan model bernama whisper_model. Fungsi ini pertama-tama memeriksa apakah flag gpu disetel ke True. Jika iya, whisper_model digunakan dengan parameter tertentu seperti word_timestamps=True, fp16=True, language='English', dan task='translate'. Jika flag gpu adalah False, whisper_model digunakan dengan fp16=False. Hasil transkripsi kemudian disimpan ke file bernama 'scan.txt' dan dikembalikan sebagai teks.

Fungsi run_text_prompt menerima message dan chat_history sebagai input. Fungsi ini menggunakan phi_demo untuk menghasilkan respons dari chatbot berdasarkan pesan yang diberikan. Respons yang dihasilkan kemudian diteruskan ke fungsi talk, yang mengubah respons tersebut menjadi file audio dan mengembalikan path file tersebut. Kelas Audio digunakan untuk menampilkan dan memutar file audio. Audio ditampilkan menggunakan fungsi display dari modul IPython.display, dan objek Audio dibuat dengan parameter autoplay=True, sehingga audio mulai diputar secara otomatis. chat_history diperbarui dengan pesan input dan respons yang dihasilkan, lalu string kosong dan chat_history yang diperbarui dikembalikan.

Kelas str adalah kelas bawaan di Python yang merepresentasikan urutan karakter. Kelas ini menyediakan berbagai metode untuk memanipulasi dan bekerja dengan string, seperti capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill, dan lainnya. Metode-metode ini memungkinkan Anda melakukan operasi seperti pencarian, penggantian, format, dan manipulasi string.

Kelas Audio adalah kelas kustom yang merepresentasikan objek audio. Kelas ini digunakan untuk membuat pemutar audio di lingkungan Jupyter Notebook. Kelas ini menerima berbagai parameter seperti data, filename, url, embed, rate, autoplay, dan normalize. Parameter data dapat berupa array numpy, daftar sampel, string yang merepresentasikan nama file atau URL, atau data PCM mentah. Parameter filename digunakan untuk menentukan file lokal yang akan dimuat data audionya, dan parameter url digunakan untuk menentukan URL untuk mengunduh data audio. Parameter embed menentukan apakah data audio harus disematkan menggunakan data URI atau direferensikan dari sumber aslinya. Parameter rate menentukan tingkat sampling data audio. Parameter autoplay menentukan apakah audio harus mulai diputar secara otomatis. Parameter normalize menentukan apakah data audio harus dinormalisasi (diskalakan ulang) ke rentang maksimum yang mungkin. Kelas Audio juga menyediakan metode seperti reload untuk memuat ulang data audio dari file atau URL, serta atribut seperti src_attr, autoplay_attr, dan element_id_attr untuk mengambil atribut yang sesuai untuk elemen audio dalam HTML.

Secara keseluruhan, fungsi dan kelas ini digunakan untuk mentranskripsi audio menjadi teks, menghasilkan respons audio dari chatbot, serta menampilkan dan memutar audio di lingkungan Jupyter Notebook.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan penerjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berusaha untuk memberikan hasil yang akurat, harap diketahui bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang otoritatif. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
